<a href="https://colab.research.google.com/github/aformen9/BC-ML-Project/blob/CHOCOOOO/BreastCancer_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para este caso de clasificación de tumores (M vs. B) sobre el dataset de Breast Cancer Wisconsin, te recomendaría emplear Regresión Logística y Random Forest. A continuación, un breve razonamiento de por qué estos dos modelos cumplen bien los requisitos del trabajo:

#Regresión Logística

- Ventajas:

Es un modelo lineal sencillo y muy interpretativo: puedes ver directamente el peso que asigna a cada variable (coeficientes), lo cual ayuda a explicar el resultado en lenguaje comprensible para un público no experto.

Suele servir como “baseline” (modelo base) para cualquier tarea de clasificación binaria; si su desempeño es razonable, sabrás que el problema no es extremadamente complejo.

Su entrenamiento es rápido y, al ser lineal, no requiere tanto ajuste de hiperparámetros.

- Aplicación en este dataset:

Permite ver claramente qué características (por ejemplo, el radio medio, la textura media, el área, etc.) tienden a asociarse más con tumores malignos o benignos.

Al graficar la matriz de confusión, podrás ver cuántos casos de cáncer benigno se clasifican correctamente (TN), cuántos falsos negativos (FN), etc., y explicarlo de forma sencilla: “de cada 100 tumores malignos, el modelo detectó X correctamente y falló en Y”.

#Random Forest

- Ventajas:

Es un ensamble de árboles de decisión que combina múltiples “árboles débiles” para obtener un clasificador robusto, capaz de capturar relaciones no lineales y jerarquías complejas entre variables.

Maneja bien la multicolinealidad y las características con diferentes escalas, y suele dar muy buena precisión en problemas de detección de enfermedades.

Permite obtener la importancia de cada variable (feature importance), lo que facilita justificar por qué ciertas medidas celulares (p. ej., concavidad o suavidad) son más relevantes para predecir malignidad.

- Aplicación en este dataset:

Podrás comparar su desempeño (accuracy, recall, precision, AUC) contra la Regresión Logística. Generalmente Random Forest supera al modelo lineal si hay interacciones o patrones complejos entre las características.

Al graficar la matriz de confusión para Random Forest, mostrarás cuántos tumores malignos detectó correctamente frente a cuántos clasificó erróneamente. Además, puedes incluir un mapa de calor de correlaciones o un boxplot/violin para justificar por qué ciertos grupos de variables (por ejemplo, medidas de perímetro o área) son marcadores fuertes y cómo Random Forest las aprovecha.

#¿Por qué no elijo Decision Tree como segundo modelo?
- Árbol de Decisión simple:

Aunque es muy interpretable y visual (puedes dibujar el árbol completo), tiende a sobreajustarse con facilidad, sobre todo en datasets con muchas variables numéricas y ruido, como este.

Al incluir Random Forest en lugar de un único árbol, te aseguras de “proteger” esa interpretación de árboles simples contra el sobreajuste, ya que el ensamble promedia múltiples árboles.

Sin embargo, podrías usar Decision Tree como experimento adicional (pues el enunciado lo permite), pero para el entregable final te alcanzan dos modelos: uno lineal (Regresión Logística) y uno no lineal/ensemble (Random Forest).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
cancer_data = pd.read_csv('/content/Breast Cancer Wisconsin_TF - Breast Cancer Wisconsin_TF.csv')
cancer_data.head(10)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN
5,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,NaN
6,844359,M,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,NaN
7,84458202,M,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,NaN
8,844981,M,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,...,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720,NaN
9,84501001,M,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,...,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750,NaN


In [ ]:
cancer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              552 non-null    float64
 3   texture_mean             541 non-null    float64
 4   perimeter_mean           513 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [ ]:
#holaaaaa